# XArrayDataset

In [1]:
import autoroot
import typing as tp
from dataclasses import dataclass
import numpy as np
import einops
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDAPatcher

%load_ext autoreload
%autoreload 2


This tutorial walks through some of the nice features of the custom `XRDAPatcher` class.
This is a custom class that slices and dices through an `xr.DataArray` where a user can specify explicitly the patch dimensions and the strides.
We preallocated the *slices* and then we can arbitrarily call the slices at will.
This is very similar to the *torch.utils.data* object except we are only working with `xr.DataArray`'s directly.


There have been other previous attempts at this, e.g. `xBatcher`.
However, we found the API very cumbersome and non-intuitive.
This is our attempt to design an API that we are comfortable with and that we find easy to use.

Below, we have outlined a few use-cases that users may be interested in. 
These use cases are:

* Chunking a 1-Dimensional Time Series
* Patch-ify a 2D Grid
* Cube-ify a 3D Volume
* Cube-ify a 2D+T Spatio-Temporal Field
* Reconstructing Multiple Variables
* Choosing Specific Dimensions for Reconstructions

We will walk through each of these and highlight how this can be achieved with the custom `XRDAPatcher` class.

## Case I: Chunking a 1D TS

In [2]:
TIME = tp.Literal["time"]

@dataclass
class TimeAxis:
    data: xrdataclass.Data[TIME, tp.Literal["datetime64[ns]"]]
    name: xrdataclass.Name[str] = "time"
    long_name: xrdataclass.Attr[str] = "Date"

@dataclass
class Variable1D:
    data: xrdataclass.Data[tuple[TIME], np.float32]
    time: xrdataclass.Coordof[TimeAxis] = 0
    name: xrdataclass.Attr[str] = "var"

In [3]:

t = np.arange(1, 360+1, 1)
rng = np.random.RandomState(seed=123)
ts = np.sin(t)

ts = Variable1D(data=ts, time=t, name="var")

da = xrdataclass.asdataarray(ts)

da

<xarray.DataArray (time: 360)>
array([ 8.41470957e-01,  9.09297407e-01,  1.41120002e-01, -7.56802499e-01,
       -9.58924294e-01, -2.79415488e-01,  6.56986594e-01,  9.89358246e-01,
        4.12118495e-01, -5.44021130e-01, -9.99990225e-01, -5.36572933e-01,
        4.20167029e-01,  9.90607381e-01,  6.50287867e-01, -2.87903309e-01,
       -9.61397469e-01, -7.50987232e-01,  1.49877205e-01,  9.12945271e-01,
        8.36655617e-01, -8.85130931e-03, -8.46220434e-01, -9.05578375e-01,
       -1.32351756e-01,  7.62558460e-01,  9.56375957e-01,  2.70905793e-01,
       -6.63633883e-01, -9.88031626e-01, -4.04037654e-01,  5.51426709e-01,
        9.99911845e-01,  5.29082716e-01, -4.28182662e-01, -9.91778851e-01,
       -6.43538117e-01,  2.96368569e-01,  9.63795364e-01,  7.45113134e-01,
       -1.58622667e-01, -9.16521549e-01, -8.31774771e-01,  1.77019257e-02,
        8.50903511e-01,  9.01788354e-01,  1.23573124e-01, -7.68254638e-01,
       -9.53752637e-01, -2.62374848e-01,  6.70229197e-01,  9.86627579e-01,
        3.95925164e-01, -5.58789074e-01, -9.99755144e-01, -5.21551013e-01,
        4.36164767e-01,  9.92872655e-01,  6.36738002e-01, -3.04810613e-01,
       -9.66117799e-01, -7.39180684e-01,  1.67355701e-01,  9.20026064e-01,
        8.26828659e-01, -2.65511535e-02, -8.55519950e-01, -8.97927701e-01,
       -1.14784814e-01,  7.73890674e-01,  9.51054633e-01,  2.53823370e-01,
       -6.76771939e-01, -9.85146284e-01, -3.87781650e-01,  5.66107631e-01,
        9.99520183e-01,  5.13978481e-01, -4.44112659e-01, -9.93888676e-01,
...
        7.73871601e-01, -1.14814758e-01, -8.97940934e-01, -8.55504394e-01,
       -2.65210196e-02,  8.26845646e-01,  9.20014203e-01,  1.67325988e-01,
       -7.39201009e-01, -9.66109991e-01, -3.04781914e-01,  6.36761248e-01,
        9.92869079e-01,  4.36137617e-01, -5.21576703e-01, -9.99755859e-01,
       -5.58764040e-01,  3.95952821e-01,  9.86632526e-01,  6.70206785e-01,
       -2.62403935e-01, -9.53761697e-01, -7.68235385e-01,  1.23603038e-01,
        9.01801348e-01,  8.50887716e-01,  1.76717862e-02, -8.31791461e-01,
       -9.16509509e-01, -1.58592910e-01,  7.45133281e-01,  9.63787377e-01,
        2.96339780e-01, -6.43561184e-01, -9.91774976e-01, -4.28155422e-01,
        5.29108286e-01,  9.99912262e-01,  5.51401556e-01, -4.04065222e-01,
       -9.88036275e-01, -6.63611352e-01,  2.70934820e-01,  9.56384718e-01,
        7.62538970e-01, -1.32381633e-01, -9.05591130e-01, -8.46204340e-01,
       -8.82116612e-03,  8.36672127e-01,  9.12932932e-01,  1.49847403e-01,
       -7.51007140e-01, -9.61389184e-01, -2.87874460e-01,  6.50310755e-01,
        9.90603209e-01,  4.20139670e-01, -5.36598384e-01, -9.99990344e-01,
       -5.43995798e-01,  4.12145942e-01,  9.89362657e-01,  6.56963885e-01,
       -2.79444456e-01, -9.58932817e-01, -7.56782770e-01,  1.41149849e-01,
        9.09309983e-01,  8.41454685e-01, -3.01443542e-05, -8.41487288e-01,
       -9.09284890e-01, -1.41090170e-01,  7.56822169e-01,  9.58915710e-01],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.000000001 ... 1970-01-...
Attributes:
    name:     var

In [4]:
# da.plot()

In this first example, we are going to do a non-overlapping style.
We will take a 30 day window with a 30 day stride.
This will give us exactly 12 patches (like 12 months).

In [5]:
patches = {"time": 30}
strides = {"time": 30}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")


XArray Patcher
DataArray size: OrderedDict([('time', 360)])
Patches:        OrderedDict([('time', 30)])
Strides:        OrderedDict([('time', 30)])
Num Batches:    OrderedDict([('time', 12)])
Dataset(size): 12


In this example, we will incorporate overlapping windows.
We will do a 30 day window but we will have a 15 day stride.
So, we have a 15 day overlap when creating the patches.
We can do the mental calculation already because it's quite simple:

$$
\text{Patches} = \frac{360 \text{ days total } - 30 \text{ day patches }}{15 \text{ day stride }} + 1
$$

If this is nicely divisible, we wont have any problems. 
However, often times it's not so we might have to use the `floor` operator to ensure we get integers
Our method will give a warning (optional) which lets the user know there is an issue.

In [6]:
patches = {"time": 30}
strides = {"time": 15}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('time', 360)])
Patches:        OrderedDict([('time', 30)])
Strides:        OrderedDict([('time', 15)])
Num Batches:    OrderedDict([('time', 23)])
Dataset(size): 23


## Case II: Patchify a 2D Grid

In [7]:
TIME = tp.Literal["time"]
X = tp.Literal["x"]
Y = tp.Literal["y"]

@dataclass
class TimeAxis:
    data: xrdataclass.Data[TIME, tp.Literal["datetime64[ns]"]]
    name: xrdataclass.Name[str] = "time"
    long_name: xrdataclass.Attr[str] = "Date"

@dataclass
class XAxis:
    data: xrdataclass.Data[X, np.float32]
    name: xrdataclass.Name[str] = "x"

@dataclass
class YAxis:
    data: xrdataclass.Data[Y, np.float32]
    name: xrdataclass.Name[str] = "y"

@dataclass
class Variable2D:
    data: xrdataclass.Data[tuple[X, Y], np.float32]
    x: xrdataclass.Coordof[XAxis] = 0
    y: xrdataclass.Coordof[YAxis] = 0
    name: xrdataclass.Attr[str] = "var"


In [8]:

x = np.linspace(-1, 1, 128)
y = np.linspace(-2, 2, 128)
rng = np.random.RandomState(seed=123)

data = rng.randn(x.shape[0], y.shape[0])

grid = Variable2D(data=data, x=x, y=y, name="var")

da = xrdataclass.asdataarray(grid)


da

<xarray.DataArray (x: 128, y: 128)>
array([[-1.08563066e+00,  9.97345448e-01,  2.82978505e-01, ...,
        -2.70232379e-01,  8.12341332e-01,  4.99740154e-01],
       [ 4.74347293e-01, -5.63923955e-01, -9.97321486e-01, ...,
        -1.69205236e+00,  7.42686212e-01, -8.05832222e-02],
       [ 5.90704203e-01,  1.15298726e-01,  2.96429247e-02, ...,
        -1.00750875e+00, -1.30578589e+00, -8.82829010e-01],
       ...,
       [ 2.91712314e-01,  2.60095268e-01,  2.03645043e-03, ...,
        -1.18166238e-01,  1.05859327e+00,  1.41544914e+00],
       [-1.15834916e+00, -1.15461178e-01, -2.37304592e+00, ...,
        -3.28264564e-01,  3.57845932e-01,  1.23318768e+00],
       [-2.53573377e-02, -4.33411151e-01,  1.04001844e+00, ...,
         1.29953909e+00,  1.35798299e+00, -5.47325552e-01]], dtype=float32)
Coordinates:
  * x        (x) float32 -1.0 -0.9843 -0.9685 -0.9528 ... 0.9685 0.9843 1.0
  * y        (y) float32 -2.0 -1.969 -1.937 -1.906 ... 1.906 1.937 1.969 2.0
Attributes:
    name:     var

In [9]:
# da.T.plot.imshow()

We will have a `[20,20]` patch with no overlap, `[20,20]`

In [10]:
patches = {"x": 8, "y": 8}
strides = {"x": 8, "y": 8}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8)])
Strides:        OrderedDict([('x', 8), ('y', 8)])
Num Batches:    OrderedDict([('x', 16), ('y', 16)])
Dataset(size): 256


We will have a `[20,20]` patch with some overlap, like the boundaries of 2, `[2,2]`

In [11]:
patches = {"x": 8, "y": 8}
strides = {"x": 2, "y": 2}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8)])
Strides:        OrderedDict([('x', 2), ('y', 2)])
Num Batches:    OrderedDict([('x', 61), ('y', 61)])
Dataset(size): 3721


## Case III: Cube-ify a 3D Volume

In [12]:
TIME = tp.Literal["time"]
X = tp.Literal["x"]
Y = tp.Literal["y"]
Z = tp.Literal["z"]
@dataclass
class TimeAxis:
    data: xrdataclass.Data[TIME, tp.Literal["datetime64[ns]"]]
    name: xrdataclass.Name[str] = "time"
    long_name: xrdataclass.Attr[str] = "Date"

@dataclass
class XAxis:
    data: xrdataclass.Data[X, np.float32]
    name: xrdataclass.Name[str] = "x"

@dataclass
class YAxis:
    data: xrdataclass.Data[Y, np.float32]
    name: xrdataclass.Name[str] = "y"

@dataclass
class ZAxis:
    data: xrdataclass.Data[Z, np.float32]
    name: xrdataclass.Name[str] = "z"

@dataclass
class Variable3D:
    data: xrdataclass.Data[tuple[X, Y, Z], np.float32]
    x: xrdataclass.Coordof[XAxis] = 0
    y: xrdataclass.Coordof[YAxis] = 0
    z: xrdataclass.Coordof[ZAxis] = 0
    name: xrdataclass.Attr[str] = "var"

In [13]:

x = np.linspace(-1, 1, 128)
y = np.linspace(-2, 2, 128)
z = np.linspace(-5, 5, 128)
rng = np.random.RandomState(seed=123)

data = rng.randn(x.shape[0], y.shape[0], z.shape[0])

grid = Variable3D(data=data, x=x, y=y, z=z, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray (x: 128, y: 128, z: 128)>
array([[[-1.08563066e+00,  9.97345448e-01,  2.82978505e-01, ...,
         -2.70232379e-01,  8.12341332e-01,  4.99740154e-01],
        [ 4.74347293e-01, -5.63923955e-01, -9.97321486e-01, ...,
         -1.69205236e+00,  7.42686212e-01, -8.05832222e-02],
        [ 5.90704203e-01,  1.15298726e-01,  2.96429247e-02, ...,
         -1.00750875e+00, -1.30578589e+00, -8.82829010e-01],
        ...,
        [ 2.91712314e-01,  2.60095268e-01,  2.03645043e-03, ...,
         -1.18166238e-01,  1.05859327e+00,  1.41544914e+00],
        [-1.15834916e+00, -1.15461178e-01, -2.37304592e+00, ...,
         -3.28264564e-01,  3.57845932e-01,  1.23318768e+00],
        [-2.53573377e-02, -4.33411151e-01,  1.04001844e+00, ...,
          1.29953909e+00,  1.35798299e+00, -5.47325552e-01]],

       [[-1.70722687e+00,  5.57956219e-01,  4.44460094e-01, ...,
          8.85603487e-01, -1.27480626e+00,  6.18890643e-01],
        [ 1.91074267e-01,  4.52165812e-01,  6.99595332e-01, ...,
         -1.65595734e+00, -1.16251850e+00,  8.45763147e-01],
        [-7.28351116e-01, -6.65624797e-01,  5.67999125e-01, ...,
          8.14727128e-01, -6.09483495e-02,  1.17496908e+00],
...
          1.92505896e-01, -1.65180552e+00, -1.42965364e+00],
        [-5.55472314e-01,  1.41632783e+00,  1.09950602e+00, ...,
         -4.10688937e-01,  9.35771346e-01,  9.78887796e-01],
        [ 1.54823375e+00, -2.76181841e+00,  1.62751520e+00, ...,
          5.96791863e-01, -4.17464137e-01,  9.73138809e-01]],

       [[-6.88130498e-01, -3.16136092e-01, -1.17485714e+00, ...,
          1.17088348e-01, -2.50292778e-01,  6.58283941e-03],
        [-8.52108955e-01,  3.78952682e-01, -7.20112622e-02, ...,
         -2.64783531e-01,  8.96649361e-02, -2.40705431e-01],
        [-1.09796715e+00,  1.59151518e+00, -1.33855128e+00, ...,
          2.12461263e-01,  7.84749258e-03,  1.63140607e+00],
        ...,
        [ 1.80939510e-01, -7.21302271e-01, -9.05182481e-01, ...,
          1.47239554e+00,  5.72573423e-01, -1.69221210e+00],
        [-1.21327706e-01,  1.69445598e+00,  2.11413622e-01, ...,
         -1.15567064e+00, -3.80903304e-01,  1.14710522e+00],
        [ 7.14538842e-02,  1.43959180e-01, -1.50091970e+00, ...,
          1.61168969e+00,  1.37786955e-01,  9.26089466e-01]]],
      dtype=float32)
Coordinates:
  * x        (x) float32 -1.0 -0.9843 -0.9685 -0.9528 ... 0.9685 0.9843 1.0
  * y        (y) float32 -2.0 -1.969 -1.937 -1.906 ... 1.906 1.937 1.969 2.0
  * z        (z) float32 -5.0 -4.921 -4.843 -4.764 ... 4.764 4.843 4.921 5.0
Attributes:
    name:     var

We will have a `[20,20]` patch with no overlap, `[20,20]`

In [14]:
patches = {"x": 8, "y": 8, "z": 8}
strides = {"x": 8, "y": 8, "z": 8}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128), ('z', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8), ('z', 8)])
Strides:        OrderedDict([('x', 8), ('y', 8), ('z', 8)])
Num Batches:    OrderedDict([('x', 16), ('y', 16), ('z', 16)])
Dataset(size): 4096


We will have a `[20,20]` patch with some overlap, like the boundaries of 2, `[2,2]`

In [15]:
patches = {"x": 8, "y": 8, "z": 8}
strides = {"x": 2, "y": 2, "z": 2}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('x', 128), ('y', 128), ('z', 128)])
Patches:        OrderedDict([('x', 8), ('y', 8), ('z', 8)])
Strides:        OrderedDict([('x', 2), ('y', 2), ('z', 2)])
Num Batches:    OrderedDict([('x', 61), ('y', 61), ('z', 61)])
Dataset(size): 226981


## Case IV: Cube-ify a 2D+T Spatio-Temporal Field

In [16]:
TIME = tp.Literal["time"]
X = tp.Literal["x"]
Y = tp.Literal["y"]
Z = tp.Literal["z"]

@dataclass
class TimeAxis:
    data: xrdataclass.Data[TIME, tp.Literal["datetime64[ns]"]]
    name: xrdataclass.Name[str] = "time"
    long_name: xrdataclass.Attr[str] = "Date"

@dataclass
class XAxis:
    data: xrdataclass.Data[X, np.float32]
    name: xrdataclass.Name[str] = "x"

@dataclass
class YAxis:
    data: xrdataclass.Data[Y, np.float32]
    name: xrdataclass.Name[str] = "y"

@dataclass
class ZAxis:
    data: xrdataclass.Data[Z, np.float32]
    name: xrdataclass.Name[str] = "z"

@dataclass
class Variable2DT:
    data: xrdataclass.Data[tuple[TIME, X, Y], np.float32]
    x: xrdataclass.Coordof[XAxis] = 0
    y: xrdataclass.Coordof[YAxis] = 0
    time: xrdataclass.Coordof[TimeAxis] = 0
    name: xrdataclass.Attr[str] = "var"

In [17]:

x = np.linspace(-1, 1, 200)
y = np.linspace(-2, 2, 200)
t = np.arange(1, 360+1, 1)
rng = np.random.RandomState(seed=123)

data = rng.randn(t.shape[0], x.shape[0], y.shape[0])

grid = Variable2DT(data=data, x=x, y=y, time=t, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray (time: 360, x: 200, y: 200)>
array([[[-1.0856307 ,  0.99734545,  0.2829785 , ..., -0.11085072,
         -0.3412617 , -0.21794626],
        [ 0.70331013, -0.5981053 ,  2.2007022 , ...,  0.2453794 ,
         -0.16842642,  0.24402694],
        [ 1.5340903 , -0.5299141 , -0.49097228, ..., -1.3943751 ,
          0.91268796, -1.2735702 ],
        ...,
        [-1.254297  , -0.15342525,  0.39214352, ...,  1.4914658 ,
         -0.13722242, -0.9896969 ],
        [ 0.9222702 ,  0.17628324,  1.4639887 , ...,  1.3550974 ,
          1.2875272 ,  0.5203795 ],
        [-0.03978635, -0.18871497, -1.5397139 , ...,  0.5256233 ,
          0.99480015, -0.80351275]],

       [[ 1.9122499 , -0.26827142, -0.77934915, ...,  0.7812284 ,
         -0.34153607,  0.5694327 ],
        [-0.0183752 , -0.12864375,  0.6058509 , ...,  1.3874226 ,
          1.0075318 , -0.14172022],
        [-0.740439  ,  1.7056271 , -1.1431531 , ..., -1.2670795 ,
         -0.02793068,  0.08372207],
...
        [-0.6960011 ,  0.14616081, -0.6671108 , ..., -0.22940315,
          1.0476565 , -1.6878732 ],
        [ 0.93809146, -1.3445573 ,  0.95054275, ...,  0.3897855 ,
          0.45997804,  0.11520644],
        [-0.23678194, -1.1773705 , -0.9713852 , ...,  1.2457405 ,
          2.7650023 ,  0.20476009]],

       [[ 0.71022624, -0.5875608 ,  0.46716788, ..., -1.9052417 ,
         -0.06033505,  1.0841712 ],
        [ 0.25029686, -1.0295818 ,  1.0811172 , ...,  0.48893106,
         -0.3933368 ,  1.3902658 ],
        [ 0.3809181 , -0.61940116,  1.0491153 , ..., -1.4824746 ,
         -0.2576864 ,  0.15616395],
        ...,
        [ 0.04154942, -1.058228  , -0.91710246, ...,  0.2711696 ,
         -0.90327495,  0.00667841],
        [-1.1869855 ,  1.4856511 ,  0.57484615, ...,  0.84998125,
         -0.66541153, -0.748355  ],
        [-0.33974022, -0.8361204 , -0.5343129 , ...,  0.8719576 ,
          0.19112131, -0.36490965]]], dtype=float32)
Coordinates:
  * x        (x) float32 -1.0 -0.9899 -0.9799 -0.9698 ... 0.9799 0.9899 1.0
  * y        (y) float32 -2.0 -1.98 -1.96 -1.94 -1.92 ... 1.94 1.96 1.98 2.0
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.000000001 ... 1970-01-...
Attributes:
    name:     var

Now, this is a rather big field.
Let's say we want to use some ML method with a CNN to learn how to predict ...
However, ingesting this large patch would be very difficult.
So instead, we will use the standard size for many CNNs, which is `[64,64]`.
In addition, we will use a temporal window of 15 days. 
So the patch will be `[15,64,64]`. 

As will the above examples, we will also account for the overlap in the spatial borders with `[4,4]` strides.
And lastly, we will have a 5 day overlap for the time steps.
So the final strides will be `[5,4,4]`

In [18]:
patches = {"x": 64, "y": 64, "time": 15}
strides = {"x": 4, "y": 4, "time": 5}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('time', 360), ('x', 200), ('y', 200)])
Patches:        OrderedDict([('time', 15), ('x', 64), ('y', 64)])
Strides:        OrderedDict([('time', 5), ('x', 4), ('y', 4)])
Num Batches:    OrderedDict([('time', 70), ('x', 35), ('y', 35)])
Dataset(size): 85750


All of the sudden, we have a LOT of data if we do things in a patch-wise manner, more than 85K samples!
However, we know from statistics that perhaps this isn't the greatest idea because there are a lot of overlap.
So we can be clever and use a training dataset with less overlap. 
However, we can create a different dataset for predictions where we reduce the strides considerably so that we take a weighted average over the predictions!

In [19]:
patches = {"x": 64, "y": 64, "time": 15}
strides = {"x": 1, "y": 1, "time": 1}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('time', 360), ('x', 200), ('y', 200)])
Patches:        OrderedDict([('time', 15), ('x', 64), ('y', 64)])
Strides:        OrderedDict([('time', 1), ('x', 1), ('y', 1)])
Num Batches:    OrderedDict([('time', 346), ('x', 137), ('y', 137)])
Dataset(size): 6494074


So this will cover use because we can take a weighted average of all of the predictions!

## Case V: Reconstructing with multiple variables

In this example, we look at how we can do reconstructions with multiple variables.
This may occur when we have used different methods to make predictions and we want to reconstruct all of them.

Another example is when we have some sort of latent variable representation and we would like to reconstruct each of the latent variable representations.


In [20]:

t = np.arange(1, 360+1, 1)
rng = np.random.RandomState(seed=123)
ts = np.sin(t)

ts = Variable1D(data=ts, time=t, name="var")

da = xrdataclass.asdataarray(ts)


In [21]:
patches = {"time": 30}
strides = {"time": 30}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")


XArray Patcher
DataArray size: OrderedDict([('time', 360)])
Patches:        OrderedDict([('time', 30)])
Strides:        OrderedDict([('time', 30)])
Num Batches:    OrderedDict([('time', 12)])
Dataset(size): 12


In [22]:
all_batches = list(map(lambda x: x.data, xrda_batches))
all_batches_latent = list(map(lambda x: einops.repeat(x, "... -> ... N", N=5), all_batches)) 

In [23]:
dims_labels = ["time", "z"]
weight = np.ones((patches["time"]))
rec_da = xrda_batches.reconstruct(all_batches_latent, dims_labels=dims_labels, weight=weight)
rec_da

100%|██████████| 12/12 [00:00<00:00, 168.59it/s]


<xarray.DataArray (time: 360, z: 5)>
array([[ 0.84147096,  0.84147096,  0.84147096,  0.84147096,  0.84147096],
       [ 0.90929741,  0.90929741,  0.90929741,  0.90929741,  0.90929741],
       [ 0.14112   ,  0.14112   ,  0.14112   ,  0.14112   ,  0.14112   ],
       ...,
       [-0.14109017, -0.14109017, -0.14109017, -0.14109017, -0.14109017],
       [ 0.75682217,  0.75682217,  0.75682217,  0.75682217,  0.75682217],
       [ 0.95891571,  0.95891571,  0.95891571,  0.95891571,  0.95891571]])
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.000000001 ... 1970-01-...
Dimensions without coordinates: z
Attributes:
    name:     var

## Case VI: Choosing a Specific Dimension for Reconstruction

In [24]:

x = np.linspace(-1, 1, 50)
y = np.linspace(-2, 2, 50)
t = np.arange(1, 30+1, 1)
rng = np.random.RandomState(seed=123)

data = rng.randn(t.shape[0], x.shape[0], y.shape[0])

grid = Variable2DT(data=data, x=x, y=y, time=t, name="var")

da = xrdataclass.asdataarray(grid)

da

<xarray.DataArray (time: 30, x: 50, y: 50)>
array([[[-1.0856307 ,  0.99734545,  0.2829785 , ...,  0.41291216,
          0.978736  ,  2.2381434 ],
        [-1.2940853 , -1.0387882 ,  1.7437122 , ..., -1.3634715 ,
          0.3794006 , -0.37917644],
        [ 0.6420547 , -1.977888  ,  0.71226466, ...,  1.5445445 ,
         -0.23966879,  0.14330773],
        ...,
        [ 0.37384272,  0.00383309,  0.09342879, ...,  0.6317589 ,
          1.0136658 ,  0.24321172],
        [-0.9554714 ,  0.5535039 ,  1.6898795 , ...,  0.08580064,
         -0.4533429 , -0.90877026],
        [-0.92812216,  0.68175596,  0.4603092 , ..., -1.6897908 ,
         -0.6358816 ,  1.6670258 ]],

       [[-1.4881413 ,  0.68179685,  0.13142663, ...,  0.6981568 ,
          2.3493557 ,  0.56583357],
        [-1.4214789 , -0.99469167, -0.9384257 , ...,  0.8125535 ,
         -0.28239277,  0.72693455],
        [ 0.08196618,  1.2273717 , -0.27881265, ..., -0.5201821 ,
         -0.52076805,  0.7228468 ],
...
        [ 0.24465546,  0.28751767,  0.4741071 , ...,  0.1838315 ,
          0.669786  , -1.173779  ],
        [-0.2496533 ,  0.40799373, -0.8288618 , ...,  0.11392058,
          1.572757  ,  0.98382896],
        [ 0.5066419 ,  0.4526176 , -1.0927416 , ..., -0.6655248 ,
         -1.0839112 , -0.08317726]],

       [[ 0.08734189, -0.4269916 , -0.48208094, ...,  0.5447468 ,
         -0.5087233 ,  1.2782396 ],
        [-0.6547844 , -0.10786748, -1.4987696 , ...,  1.6393518 ,
         -0.64821076, -0.03500866],
        [-0.44025877, -0.37050143, -0.45279846, ...,  1.2072855 ,
         -0.4181824 , -0.30565566],
        ...,
        [ 1.2096492 , -0.32449156,  1.5628148 , ..., -1.6145383 ,
          0.40049618, -1.0560101 ],
        [-1.0765294 ,  1.9652433 , -2.29441   , ..., -0.7553825 ,
          3.3833566 , -0.8276102 ],
        [ 0.7194959 ,  1.0137612 ,  1.2893164 , ..., -0.47385272,
         -0.6136846 ,  0.8182852 ]]], dtype=float32)
Coordinates:
  * x        (x) float32 -1.0 -0.9592 -0.9184 -0.8776 ... 0.9184 0.9592 1.0
  * y        (y) float32 -2.0 -1.918 -1.837 -1.755 ... 1.755 1.837 1.918 2.0
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.000000001 ... 1970-01-...
Attributes:
    name:     var

Now, this is a rather big field.
Let's say we want to use some ML method with a CNN to learn how to predict ...
However, ingesting this large patch would be very difficult.
So instead, we will use the standard size for many CNNs, which is `[64,64]`.
In addition, we will use a temporal window of 15 days. 
So the patch will be `[15,64,64]`. 

As will the above examples, we will also account for the overlap in the spatial borders with `[4,4]` strides.
And lastly, we will have a 5 day overlap for the time steps.
So the final strides will be `[5,4,4]`

In [25]:
patches = {"x": 10, "y": 10, "time": 5}
strides = {"x": 8, "y": 8, "time": 1}
domain_limits = None#{"lat": slice(-10, 10)}
check_full_scan = True

xrda_batches = XRDAPatcher(
    da=da,
    patches=patches,
    strides=strides,
    check_full_scan=check_full_scan
)

print(xrda_batches)
print(f"Dataset(size): {len(xrda_batches)}")

XArray Patcher
DataArray size: OrderedDict([('time', 30), ('x', 50), ('y', 50)])
Patches:        OrderedDict([('time', 5), ('x', 10), ('y', 10)])
Strides:        OrderedDict([('time', 1), ('x', 8), ('y', 8)])
Num Batches:    OrderedDict([('time', 26), ('x', 6), ('y', 6)])
Dataset(size): 936


Here, we can reconstruct just the time series. 
So in this case, we will take the mean of all of the spatial values x,y and just have the time series. 


In [26]:
all_batches = list(map(lambda x: x.mean(dim=["x", "y"]).data, xrda_batches))

But we still want to reconstruct! 
So we can pass these through the reconstruction but pay careful attention to the dimension we wish to reconstruct.

In [27]:
dims_labels = ["time"]
weight = np.ones((patches["time"]))
rec_da = xrda_batches.reconstruct(all_batches, dims_labels=dims_labels, weight=weight)
rec_da

100%|██████████| 936/936 [00:04<00:00, 194.67it/s]


<xarray.DataArray (time: 30)>
array([-0.00701476,  0.04654715,  0.01288853, -0.00142144,  0.04440702,
        0.00534748,  0.00940945, -0.02828223, -0.00664724, -0.02007576,
        0.00514503,  0.0243942 ,  0.01571643,  0.00943206,  0.012806  ,
        0.0090334 , -0.03173277,  0.03098274, -0.05859589,  0.05266089,
        0.00200994,  0.01722661, -0.02239833, -0.01689799, -0.00966568,
        0.00867594, -0.01743928,  0.02804271, -0.0087162 , -0.01741343])
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T00:00:00.000000001 ... 1970-01-...
Attributes:
    name:     var

Here, we can reconstruct just the x,y patches. 
So in this case, we will take the mean of all of the temporal coordinates and just have the spatial patches. 


In [28]:
all_batches = list(map(lambda x: x.mean(dim=["time"]).data, xrda_batches))

Again, we still want to reconstruct! 
So, like above, we can simply pass the correct dimensions to the reconstruction.

In [29]:
dims_labels = ["x", "y"]
weight = np.ones((patches["x"], patches["y"]))
rec_da = xrda_batches.reconstruct(all_batches, dims_labels=dims_labels, weight=weight)
rec_da

100%|██████████| 936/936 [00:07<00:00, 124.35it/s]


<xarray.DataArray (x: 50, y: 50)>
array([[ 0.46374542,  0.05509733,  0.05255119, ..., -0.06513734,
        -0.03144977, -0.09354143],
       [-0.12045896, -0.09441399,  0.12395332, ...,  0.0054683 ,
         0.07519625, -0.21840822],
       [ 0.14952884,  0.44404164, -0.24888576, ..., -0.13306253,
         0.03267373,  0.3056222 ],
       ...,
       [ 0.3156216 ,  0.15755736, -0.19105805, ..., -0.03394484,
         0.22196276,  0.00809461],
       [-0.23942302, -0.2357427 , -0.27109791, ..., -0.20633146,
         0.27118453, -0.00237496],
       [ 0.1262538 , -0.02112339,  0.058605  , ..., -0.21492687,
         0.1663647 , -0.01246316]])
Coordinates:
  * x        (x) float32 -1.0 -0.9592 -0.9184 -0.8776 ... 0.9184 0.9592 1.0
  * y        (y) float32 -2.0 -1.918 -1.837 -1.755 ... 1.755 1.837 1.918 2.0
Attributes:
    name:     var